In [36]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
pd.set_option('display.max_columns', None)

data = pd.read_csv('NBA_player_data.csv')

In [50]:
data.sample(10)

,Year,Season_type,PLAYER_ID,PLAYER,TEAM_ID,TEAM,GP,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PTS,season_start_year
3821,2020-21,RS,203901,Elfrid Payton,1610612752,NYK,63,23.6,4.3,9.9,0.432,0.4,1.6,0.286,1.2,1.7,0.682,1.0,2.4,3.4,3.2,0.7,0.1,1.6,10.1,2020
4761,2022-23,Playoffs,1628960,Grayson Allen,1610612749,MIL,5,29.8,3.8,8.2,0.463,2.8,5.8,0.483,1.2,1.4,0.857,0.2,2.2,2.4,1.8,0.4,0.0,0.8,11.6,2022
4373,2021-22,Playoffs,202326,DeMarcus Cousins,1610612743,DEN,5,11.4,3.8,5.8,0.655,0.8,1.2,0.667,2.2,3.0,0.733,0.6,2.8,3.4,1.2,0.6,0.2,1.4,10.6,2021
3051,2018-19,RS,202722,Davis Bertans,1610612759,SAS,76,21.5,2.7,6.0,0.450,1.9,4.4,0.429,0.7,0.8,0.883,0.3,3.2,3.5,1.3,0.5,0.4,0.6,8.0,2018
2896,2018-19,RS,200746,LaMarcus Aldridge,1610612759,SAS,81,33.2,8.4,16.3,0.519,0.1,0.5,0.238,4.3,5.1,0.847,3.1,6.1,9.2,2.4,0.5,1.3,1.8,21.3,2018
2098,2015-16,Playoffs,203090,Maurice Harkless,1610612757,POR,11,24.7,4.3,10.0,0.427,1.4,4.0,0.341,1.1,2.3,0.480,2.0,3.1,5.1,0.6,0.9,0.3,1.0,11.0,2015
1699,2014-15,Playoffs,202711,Bojan Bogdanovic,1610612751,BKN,6,34.3,3.8,9.8,0.390,1.8,5.5,0.333,0.8,1.2,0.714,0.5,3.3,3.8,1.7,0.7,0.3,1.3,10.3,2014
1474,2014-15,RS,2403,Nene,1610612764,WAS,67,25.3,4.6,9.0,0.511,0.0,0.1,0.200,1.8,3.0,0.604,1.3,3.9,5.1,1.8,1.0,0.3,1.9,11.0,2014
2206,2016-17,RS,2544,LeBron James,1610612739,CLE,74,37.8,9.9,18.2,0.548,1.7,4.6,0.363,4.8,7.2,0.674,1.3,7.3,8.6,8.7,1.2,0.6,4.1,26.4,2016
1081,2013-14,RS,201941,Jordan Hill,1610612747,LAL,72,20.8,4.0,7.2,0.549,0.0,0.0,0.000,1.7,2.5,0.685,2.7,4.7,7.4,0.8,0.4,0.9,1.0,9.7,2013


In [39]:
data.shape

(4847, 27)

# DATA CLEANING AND ANALYSIS

In [41]:
data.drop(columns=['RANK','EFF'], inplace=True)


In [42]:
data['season_start_year'] =data['Year'].str[:4].astype(int)

In [43]:
data['TEAM'].replace(to_replace=['NOP','NOH'], value='NO', inplace=True)

In [44]:
data['Season_type'].replace('Regular%20Season', 'RS', inplace=True)

In [45]:
rs_df = data[data['Season_type']=='RS']
playoffs_df = data[data['Season_type']=='Playoffs']

In [48]:
total_cols = ['MIN','FGM','FGA','FG3M','FG3A','FTM','FTA',
              'OREB','DREB','REB','AST','STL','BLK','TOV','PTS']

# Which player stats are correlated with each other?

In [49]:
data_per_min = data.groupby(['PLAYER','PLAYER_ID','Year'])[total_cols].sum().reset_index()
for col in data_per_min.columns[4:]:
    data_per_min[col] = data_per_min[col]/data_per_min['MIN']

data_per_min['FG%'] = data_per_min['FGM']/data_per_min['FGA']
data_per_min['3PT%'] = data_per_min['FG3M']/data_per_min['FG3A']
data_per_min['FT%'] = data_per_min['FTM']/data_per_min['FTA']
data_per_min['FG3A%'] = data_per_min['FG3A']/data_per_min['FGA']
data_per_min['PTS/FGA'] = data_per_min['PTS']/data_per_min['FGA']
data_per_min['FG3M/FGM'] = data_per_min['FG3M']/data_per_min['FGM']
data_per_min['FTA/FGA'] = data_per_min['FTA']/data_per_min['FGA']
data_per_min['TRU%'] = 0.5*data_per_min['PTS']/(data_per_min['FGA']+0.475*data_per_min['FTA'])
data_per_min['AST_TOV'] = data_per_min['AST']/data_per_min['TOV']

data_per_min = data_per_min[data_per_min['MIN']>=50]
data_per_min.drop(columns='PLAYER_ID', inplace=True)

fig = px.imshow(data_per_min.corr())
fig.show()

C:\Users\colli\AppData\Local\Temp\ipykernel_2588\3632397438.py:18: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  fig = px.imshow(data_per_min.corr())


# How many minutes played distributed?

In [60]:
fig = px.histogram(x=playoffs_df['MIN'], histnorm='percent')
fig.show()

In [ ]:
fig

# How has the game changed over the past 10 Years?